<a href="https://colab.research.google.com/github/LordRelentless/Order-Reconstruction-Data-Analysis-Challenge-Nth-Mathematics/blob/main/Order_Reconstruction_Data_Analysis_Take_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Re-Evaluation With Nth Mathematics and Nth Grover Style Ordering corrections

In [4]:
import pandas as pd

# Read one of the files to inspect its columns
try:
    df_sample = pd.read_csv('/content/file_01.csv')
    print(df_sample.columns)
except FileNotFoundError:
    print("file_01.csv not found. Please ensure the data files are in the /content directory.")
except Exception as e:
    print(f"An error occurred while reading the sample file: {e}")

Index(['v', 'zct'], dtype='object')


In [14]:
# Colab-ready full pipeline: v/zct -> features -> anomaly clusters -> bell-curve order -> final submission
import os, glob, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis
from scipy.signal import butter, filtfilt

# ============================
# Configuration
# ============================
DATA_DIR = "/content"       # expects file_01.csv ... file_53.csv with columns ['v','zct']
FS = 93750.0                # acceleration sampling frequency (Hz)
GEAR_RATIO = 5.095238095    # tachometer shaft to turbine shaft
MEAN_TACH = 105.25          # Hz approx
NOM_TURBINE_SPEED = MEAN_TACH * GEAR_RATIO  # ~536.27 Hz
GEOMETRY_FACTORS = {"cage":0.43, "ball":7.05, "inner":10.78, "outer":8.22}
FAULT_BANDS = {k: NOM_TURBINE_SPEED*v for k,v in GEOMETRY_FACTORS.items()}
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Plot directories
PLOT_DIR_PRESENTED = os.path.join(DATA_DIR, "plots_presented_order")
PLOT_DIR_BELL = os.path.join(DATA_DIR, "plots_bell_order")
os.makedirs(PLOT_DIR_PRESENTED, exist_ok=True)
os.makedirs(PLOT_DIR_BELL, exist_ok=True)

# ============================
# Robust helpers
# ============================
def mad(x):
    x = np.asarray(x, dtype=float)
    med = np.median(x)
    return np.median(np.abs(x - med)) + 1e-12

def zscore(x, robust=False):
    x = np.asarray(x, dtype=float)
    if robust:
        mu = np.median(x)
        sigma = 1.4826 * mad(x)
    else:
        mu = np.mean(x)
        sigma = np.std(x) + 1e-12
    return (x - mu) / sigma

def gaussian_smooth(arr, window, sigma=None):
    arr = np.asarray(arr, dtype=float)
    w = int(max(3, window))
    if sigma is None:
        sigma = max(1.0, w / 5.0)
    half = w // 2
    x = np.arange(-half, half + 1, dtype=float)
    ker = np.exp(-(x**2) / (2 * sigma**2))
    ker = ker / (ker.sum() + 1e-12)
    pad = np.pad(arr, (half, half), mode='edge')
    sm = np.convolve(pad, ker, mode='valid')
    return sm

def smooth_centered(arr, window, method="gaussian"):
    if method == "gaussian":
        return gaussian_smooth(arr, window)
    else:
        return pd.Series(arr).rolling(int(window), center=True, min_periods=1).mean().values

# ============================
# Loader and zct cleaning
# ============================
def load_files(data_dir=DATA_DIR):
    files = sorted(glob.glob(os.path.join(data_dir,"file_*.csv")))
    items = {}
    for path in files:
        base = os.path.basename(path)
        m = re.match(r"file_(\d+)\.csv", base)
        if not m:
            continue
        item_id = int(m.group(1))
        try:
            df = pd.read_csv(path)
        except Exception as e:
            print(f"Skip {base}: read error {e}")
            continue
        if not {"v","zct"}.issubset(df.columns):
            print(f"Skip {base}: missing v/zct")
            continue
        # numeric coercion
        df = df[["v","zct"]].apply(pd.to_numeric, errors="coerce")
        # drop v NaNs; keep zct for cleaning
        df = df.dropna(subset=["v"]).copy()
        zct = df["zct"].dropna().values
        # clean zct: sort, unique, rolling median, enforce monotonicity
        if len(zct) >= 2:
            zct = np.sort(zct)
            zct = np.unique(zct)
            if len(zct) >= 5:
                zct = pd.Series(zct).rolling(5, center=True, min_periods=1).median().values
            diffs = np.diff(zct)
            idx_keep = np.where(diffs > 0)[0]
            zct_clean = [zct[0]]
            for idx in idx_keep:
                zct_clean.append(zct[idx+1])
            zct = np.array(zct_clean, dtype=float)
        if len(zct) < 2:
            # fallback: simulate sparse cycle boundaries across the recording
            # Use 1-second spacing as a crude fallback (FS known), up to signal length
            n_samples = len(df["v"])
            approx_cycles = int(n_samples / FS) - 1
            if approx_cycles <= 1:
                print(f"Skip {base}: insufficient zct and too few samples")
                continue
            zct = np.linspace(0.0, approx_cycles, approx_cycles+1)
        items[item_id] = {"v": df["v"].values.astype(float), "zct": zct.astype(float), "filename": base}
    print(f"Loaded {len(items)} files")
    return items

# ============================
# Tachometer/turbine utilities
# ============================
def tachometer_frequency(zct):
    delta = np.diff(zct)
    delta = np.where(delta <= 0, np.nan, delta)
    f = 1.0/(delta)
    return f

def turbine_speed(f_tach):
    return f_tach * GEAR_RATIO

# ============================
# Band energy feature
# ============================
def band_energy(signal, fs, center, bw=50.0):
    nyq = 0.5*fs
    low_hz = max(1.0, center - bw)
    high_hz = min(nyq - 1.0, center + bw)
    if low_hz >= high_hz:
        return 0.0
    low = low_hz/nyq
    high = high_hz/nyq
    try:
        b,a = butter(4,[low,high],btype='band')
        filt = filtfilt(b,a,signal)
        return float(np.mean(filt**2))
    except Exception:
        return 0.0

# ============================
# Cycle segmentation and engineered features
# ============================
def extract_cycle_features(v, zct, fs=FS):
    v = np.asarray(v, dtype=float)
    zct = np.asarray(zct, dtype=float)
    delta = np.diff(zct)
    f_tach_series = tachometer_frequency(zct)
    turbine_series = turbine_speed(f_tach_series)
    cycles = []
    for i in range(len(zct)-1):
        if i >= len(f_tach_series) or not np.isfinite(f_tach_series[i]) or delta[i] <= 0:
            continue
        start_f = zct[i] * fs
        end_f   = zct[i+1] * fs
        if not np.isfinite(start_f) or not np.isfinite(end_f):
            continue
        start = int(max(0, min(len(v)-1, np.floor(start_f))))
        end   = int(max(0, min(len(v),   np.floor(end_f))))
        if end <= start or (end - start) < 20:
            continue
        seg = v[start:end]
        feats = {
            "cycle_index": i+1,
            "rms": float(np.sqrt(np.mean(seg**2))),
            "skew": float(skew(seg)),
            "kurt": float(kurtosis(seg)),
            "tach_mean": float(f_tach_series[i]) if np.isfinite(f_tach_series[i]) else 0.0,
            "turbine_mean": float(turbine_series[i]) if np.isfinite(turbine_series[i]) else 0.0,
        }
        feats["band_cage"]  = band_energy(seg, fs, FAULT_BANDS["cage"],  bw=40.0)
        feats["band_ball"]  = band_energy(seg, fs, FAULT_BANDS["ball"],  bw=80.0)
        feats["band_inner"] = band_energy(seg, fs, FAULT_BANDS["inner"], bw=80.0)
        feats["band_outer"] = band_energy(seg, fs, FAULT_BANDS["outer"], bw=80.0)
        cycles.append(feats)
    return pd.DataFrame(cycles)

def process_all(data_dir=DATA_DIR):
    items = load_files(data_dir)
    processed = {}
    for item_id, obj in items.items():
        feats = extract_cycle_features(obj["v"], obj["zct"], fs=FS)
        processed[item_id] = feats
        print(f"Item {item_id:02d} ({obj['filename']}): {len(feats)} cycles extracted")
    return items, processed

# ============================
# Graphing: presented order and bell-curve order
# ============================
def plot_factors_presented(items, processed):
    # Presented order: file_01 -> file_53 by numeric id
    for item_id in sorted(items.keys()):
        v = items[item_id]["v"]
        zct = items[item_id]["zct"]
        df = processed[item_id]
        # figure with v and zct markers, cycle boundaries
        fig, ax = plt.subplots(2, 1, figsize=(10,6), sharex=False)
        ax[0].plot(v, color='steelblue', lw=0.8)
        ax[0].set_title(f"Item {item_id:02d} - v (acceleration)")
        # mark zct positions in sample index
        zct_samples = (zct * FS).astype(int)
        zct_samples = zct_samples[(zct_samples >= 0) & (zct_samples < len(v))]
        ax[0].vlines(zct_samples, ymin=np.min(v), ymax=np.max(v), color='orange', alpha=0.3, lw=0.5)
        # cycle index vs RMS
        if not df.empty:
            ax[1].plot(df["cycle_index"], df["rms"], 'o-', color='darkred', lw=1.0, ms=3)
            ax[1].set_title("Cycle RMS vs cycle_index")
            ax[1].set_xlabel("cycle_index")
            ax[1].set_ylabel("RMS")
        else:
            ax[1].text(0.5, 0.5, "No cycles extracted", ha='center', va='center')
        fig.tight_layout()
        fig.savefig(os.path.join(PLOT_DIR_PRESENTED, f"item_{item_id:02d}_presented.png"))
        plt.close(fig)

def bell_curve_scalar(df):
    # Use per-item scalar based on normalized mean RMS and band energies
    if df.empty:
        return 0.0
    rms_mean = float(np.mean(df["rms"]))
    be_mean = float(np.mean(df[["band_cage","band_ball","band_inner","band_outer"]].sum(axis=1)))
    # Normalize to approximate Gaussian across items later via z-scoring
    return rms_mean + 0.2 * be_mean

def build_bell_curve_order(processed):
    # Compute scalar per item, then z-score across items
    scores = {}
    for item_id, df in processed.items():
        scores[item_id] = bell_curve_scalar(df)
    vals = np.array(list(scores.values()), dtype=float)
    z = zscore(vals, robust=True)
    # Construct an order that spreads items along the bell curve (ascending z)
    order = [item for item, _ in sorted(scores.items(), key=lambda kv: kv[1])]
    return order, scores, z

def plot_factors_bell(items, processed, bell_order):
    for item_id in bell_order:
        v = items[item_id]["v"]
        zct = items[item_id]["zct"]
        df = processed[item_id]
        fig, ax = plt.subplots(2, 1, figsize=(10,6), sharex=False)
        ax[0].plot(v, color='seagreen', lw=0.8)
        ax[0].set_title(f"Item {item_id:02d} - v (acceleration) [bell-curve order]")
        zct_samples = (zct * FS).astype(int)
        zct_samples = zct_samples[(zct_samples >= 0) & (zct_samples < len(v))]
        ax[0].vlines(zct_samples, ymin=np.min(v), ymax=np.max(v), color='purple', alpha=0.3, lw=0.5)
        if not df.empty:
            ax[1].plot(df["cycle_index"], df["rms"], 'o-', color='navy', lw=1.0, ms=3)
            ax[1].set_title("Cycle RMS vs cycle_index [bell-curve order]")
            ax[1].set_xlabel("cycle_index")
            ax[1].set_ylabel("RMS")
        else:
            ax[1].text(0.5, 0.5, "No cycles extracted", ha='center', va='center')
        fig.tight_layout()
        fig.savefig(os.path.join(PLOT_DIR_BELL, f"item_{item_id:02d}_bell.png"))
        plt.close(fig)

# ============================
# Bearing-feature output (per item)
# ============================
def summarize_bearing_features(processed):
    rows = []
    for item_id, df in processed.items():
        if df.empty:
            rows.append({"item_id": item_id, "rms_mean": 0.0, "skew_mean": 0.0, "kurt_mean": 0.0,
                         "tach_mean": 0.0, "turbine_mean": 0.0,
                         "band_cage_mean": 0.0, "band_ball_mean": 0.0, "band_inner_mean": 0.0, "band_outer_mean": 0.0})
            continue
        rows.append({
            "item_id": item_id,
            "rms_mean": float(np.mean(df["rms"])),
            "skew_mean": float(np.mean(df["skew"])),
            "kurt_mean": float(np.mean(df["kurt"])),
            "tach_mean": float(np.mean(df["tach_mean"])),
            "turbine_mean": float(np.mean(df["turbine_mean"])),
            "band_cage_mean": float(np.mean(df["band_cage"])),
            "band_ball_mean": float(np.mean(df["band_ball"])),
            "band_inner_mean": float(np.mean(df["band_inner"])),
            "band_outer_mean": float(np.mean(df["band_outer"])),
        })
    bf = pd.DataFrame(rows).sort_values("item_id")
    bf.to_csv(os.path.join(DATA_DIR, "bearing_features.csv"), index=False)
    return bf

# ============================
# Anomaly clustering and initiation indices (emulating 134288)
# ============================
def dynamic_window_candidates(n):
    base = max(7, n // 30)
    candidates = [
        (base, int(base*2), int(base*3)),
        (base+2, int(base*2)+4, int(base*3)+6),
        (base+4, int(base*2)+2, int(base*3)+10),
        (base, int(base*2)+6, int(base*3)+12),
        (max(5, base-2), int(base*2), int(base*3)+8),
    ]
    uniq, seen = [], set()
    for t in candidates:
        t2 = tuple(sorted([int(max(5, w)) for w in t]))
        if t2 not in seen:
            uniq.append(t2); seen.add(t2)
    return uniq

def compute_feature_anomaly(df, feature_cols):
    # anomaly per cycle via robust z of features
    if df.empty:
        return np.array([0.0])
    A = np.zeros(len(df), dtype=float)
    for col in feature_cols:
        series = df[col].astype(float).values
        A += np.abs(zscore(series, robust=True))
    return A

def initiation_index_multiscale(df, feature_cols, ignore_frac=0.07, min_run=6, tail_ignore_frac=0.0):
    if df.empty:
        return int(10**9)
    cycles = np.array(sorted(df["cycle_index"].values))
    base = compute_feature_anomaly(df, feature_cols)
    votes = []
    for windows in dynamic_window_candidates(len(cycles)):
        smoothed = []
        for w in windows:
            sm = smooth_centered(base, w, method="gaussian")
            sm_z = zscore(sm, robust=True)
            smoothed.append(sm_z)
        weights = np.array([1.0 / np.sqrt(w) for w in windows])
        weights = weights / (weights.sum() + 1e-12)
        anomaly_ms = np.tensordot(weights, np.vstack(smoothed), axes=(0,0))
        baseline_len = max(5, len(anomaly_ms) // 5)
        baseline = anomaly_ms[:baseline_len]
        thresh = float(np.mean(baseline) + 2.0 * np.std(baseline))
        slopes = np.diff(anomaly_ms)
        slope_mu = float(np.mean(slopes)) if len(slopes) else 0.0
        slope_sigma = float(np.std(slopes)) + 1e-12
        slope_gate = slope_mu + 0.8 * slope_sigma
        start_idx = int(len(cycles) * ignore_frac)
        end_idx = len(cycles) - int(len(cycles) * tail_ignore_frac)
        end_idx = max(end_idx, start_idx + min_run + 1)
        above = anomaly_ms > thresh
        candidates = []
        for i in range(start_idx, end_idx - min_run):
            sustained = np.all(above[i:i + min_run])
            slope_ok = np.all(slopes[i:i + min_run - 1] > slope_gate) if (i + min_run - 1) <= len(slopes) else False
            if sustained and slope_ok:
                local_strength = float(np.sum(slopes[i:i + min_run - 1])) if (i + min_run - 1) <= len(slopes) else 0.0
                candidates.append((cycles[i], local_strength))
        # add top-3 anomaly indices as backup votes
        topk = min(3, len(anomaly_ms))
        top_idx = np.argsort(anomaly_ms)[-topk:]
        for tidx in top_idx:
            s_local = float(slopes[tidx-1]) if tidx-1 >= 0 and tidx-1 < len(slopes) else 0.0
            candidates.append((cycles[tidx], max(0.0, s_local)))
        if not candidates:
            votes.append(int(cycles[np.argmax(anomaly_ms)]))
        else:
            c_cycles = np.array([c[0] for c in candidates])
            c_weights = np.array([c[1] for c in candidates]) + 1e-6
            c_weights = c_weights / (c_weights.sum() + 1e-12)
            order = np.argsort(c_cycles)
            cc = c_cycles[order]
            ww = c_weights[order]
            cumw = np.cumsum(ww)
            idx = np.searchsorted(cumw, 0.5)
            votes.append(int(cc[min(idx, len(cc)-1)]))
    return int(np.median(votes)) if len(votes) else int(10**9)

def anomaly_cluster_order(processed):
    feature_cols = ["rms","skew","kurt","band_cage","band_ball","band_inner","band_outer"]
    init_map = {}
    for item_id, df in processed.items():
        idx = initiation_index_multiscale(df, feature_cols, ignore_frac=0.07, min_run=6, tail_ignore_frac=0.0)
        init_map[item_id] = idx
    order = sorted(init_map.keys(), key=lambda k: (init_map[k], k))
    return order, init_map

# ============================
# Pairwise mapping (similarity/distance)
# ============================
def item_feature_vector(df):
    if df.empty:
        return np.zeros(8, dtype=float)
    vec = [
        np.mean(df["rms"]),
        np.mean(df["skew"]),
        np.mean(df["kurt"]),
        np.mean(df["tach_mean"]),
        np.mean(df["turbine_mean"]),
        np.mean(df["band_cage"]),
        np.mean(df["band_ball"]),
        np.mean(df["band_inner"]),
        # to keep vector length consistent, combine outer too
    ]
    vec.append(np.mean(df["band_outer"]))
    return np.array(vec, dtype=float)

def pairwise_map(processed):
    ids = sorted(processed.keys())
    M = np.zeros((len(ids), len(ids)), dtype=float)
    vectors = {i: item_feature_vector(processed[i]) for i in ids}
    # distance: robust z, then L1
    all_vecs = np.vstack([vectors[i] for i in ids])
    all_vecs_z = (all_vecs - np.median(all_vecs, axis=0)) / (1.4826 * (np.median(np.abs(all_vecs - np.median(all_vecs, axis=0)), axis=0) + 1e-12))
    for a_idx, ia in enumerate(ids):
        for b_idx, ib in enumerate(ids):
            M[a_idx, b_idx] = np.sum(np.abs(all_vecs_z[a_idx] - all_vecs_z[b_idx]))
    # export
    dfM = pd.DataFrame(M, index=ids, columns=ids)
    dfM.to_csv(os.path.join(DATA_DIR, "pairwise_map.csv"))
    return dfM

# ============================
# Final order fusion: bell-curve + anomaly clusters + pairwise consensus
# ============================
def ranks_from_order(order):
    return {item_id: rank for rank, item_id in enumerate(order, start=1)}

def fuse_orders(bell_order, anomaly_order, pairwise_df):
    ids = sorted(set(bell_order).union(set(anomaly_order)))
    r_bell = ranks_from_order(bell_order)
    r_anom = ranks_from_order(anomaly_order)
    # pairwise centrality: items with lower average distance are "central"
    avg_dist = {i: float(np.mean(pairwise_df.loc[i].values)) if i in pairwise_df.index else 0.0 for i in ids}
    # normalize to ranks (lower distance -> earlier)
    dist_order = [i for i, _ in sorted(avg_dist.items(), key=lambda kv: kv[1])]
    r_dist = ranks_from_order(dist_order)
    # Weighted fusion to emulate 134288 (favor anomaly, then bell, then pairwise)
    fused_score = {i: 0.55 * r_anom.get(i, len(ids)) + 0.35 * r_bell.get(i, len(ids)) + 0.10 * r_dist.get(i, len(ids)) for i in ids}
    final = [i for i, _ in sorted(fused_score.items(), key=lambda kv: kv[1])]
    return final

# ============================
# Main pipeline
# ============================
def run_pipeline_and_submit(data_dir=DATA_DIR, out_path="submission.csv"):
    # 1) Load raw files and extract features
    items, processed = process_all(data_dir)
    if not processed:
        raise RuntimeError("No items processed. Ensure /content has file_XX.csv with v,zct.")

    # 2) Graph presented order
    plot_factors_presented(items, processed)

    # 3) Bell-curve order from scalar scores
    bell_order, bell_scores, bell_z = build_bell_curve_order(processed)
    # Graph in bell order
    plot_factors_bell(items, processed, bell_order)

    # 4) Bearing-feature output
    bf = summarize_bearing_features(processed)
    print("Saved bearing_features.csv")

    # 5) Anomaly-clustered order (emulating 134288 clustering)
    anomaly_order, init_map = anomaly_cluster_order(processed)

    # 6) Pairwise mapping
    dfM = pairwise_map(processed)
    print("Saved pairwise_map.csv")

    # 7) Fuse orders into final prediction
    final_order = fuse_orders(bell_order, anomaly_order, dfM)

    # 8) Write submission
    sub_df = pd.DataFrame({"prediction": final_order})
    sub_df.to_csv(out_path, index=False)
    print(f"Wrote: {out_path}")
    print("First 20 predictions:", final_order[:20])

# ============================
# Entrypoint
# ============================
if __name__ == "__main__":
    run_pipeline_and_submit(DATA_DIR, out_path="submission.csv")

Loaded 53 files
Item 01 (file_01.csv): 211 cycles extracted
Item 02 (file_02.csv): 211 cycles extracted
Item 03 (file_03.csv): 211 cycles extracted
Item 04 (file_04.csv): 210 cycles extracted
Item 05 (file_05.csv): 211 cycles extracted
Item 06 (file_06.csv): 210 cycles extracted
Item 07 (file_07.csv): 211 cycles extracted
Item 08 (file_08.csv): 210 cycles extracted
Item 09 (file_09.csv): 210 cycles extracted
Item 10 (file_10.csv): 210 cycles extracted
Item 11 (file_11.csv): 211 cycles extracted
Item 12 (file_12.csv): 210 cycles extracted
Item 13 (file_13.csv): 211 cycles extracted
Item 14 (file_14.csv): 211 cycles extracted
Item 15 (file_15.csv): 210 cycles extracted
Item 16 (file_16.csv): 211 cycles extracted
Item 17 (file_17.csv): 210 cycles extracted
Item 18 (file_18.csv): 211 cycles extracted
Item 19 (file_19.csv): 210 cycles extracted
Item 20 (file_20.csv): 210 cycles extracted
Item 21 (file_21.csv): 211 cycles extracted
Item 22 (file_22.csv): 211 cycles extracted
Item 23 (file_23

Pair-Wise mapping of V vs ZCT

In [15]:
# Colab-ready: relational graphs across files, pairwise V–ZCT mapping, bell-curve ordering, anomaly clustering, and final prediction
import os, glob, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis
from scipy.signal import butter, filtfilt

# ============================
# Configuration
# ============================
DATA_DIR = "/content"       # expects file_01.csv ... file_53.csv with columns ['v','zct']
FS = 93750.0                # acceleration sampling frequency (Hz)
GEAR_RATIO = 5.095238095    # tachometer shaft to turbine shaft
MEAN_TACH = 105.25          # Hz approx
NOM_TURBINE_SPEED = MEAN_TACH * GEAR_RATIO  # ~536.27 Hz
GEOMETRY_FACTORS = {"cage":0.43, "ball":7.05, "inner":10.78, "outer":8.22}
FAULT_BANDS = {k: NOM_TURBINE_SPEED*v for k,v in GEOMETRY_FACTORS.items()}

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Plot output directories
PLOT_DIR = os.path.join(DATA_DIR, "relational_graphs")
os.makedirs(PLOT_DIR, exist_ok=True)

# ============================
# Helpers (robust stats, smoothing)
# ============================
def mad(x):
    x = np.asarray(x, dtype=float)
    med = np.median(x)
    return np.median(np.abs(x - med)) + 1e-12

def zscore(x, robust=False):
    x = np.asarray(x, dtype=float)
    if robust:
        mu = np.median(x)
        sigma = 1.4826 * mad(x)
    else:
        mu = np.mean(x)
        sigma = np.std(x) + 1e-12
    return (x - mu) / sigma

def gaussian_smooth(arr, window, sigma=None):
    arr = np.asarray(arr, dtype=float)
    w = int(max(3, window))
    if sigma is None:
        sigma = max(1.0, w / 5.0)
    half = w // 2
    x = np.arange(-half, half + 1, dtype=float)
    ker = np.exp(-(x**2) / (2 * sigma**2))
    ker = ker / (ker.sum() + 1e-12)
    pad = np.pad(arr, (half, half), mode='edge')
    sm = np.convolve(pad, ker, mode='valid')
    return sm

def smooth_centered(arr, window=9):
    return gaussian_smooth(arr, window)

# ============================
# Loader and zct cleaning
# ============================
def load_files(data_dir=DATA_DIR):
    files = sorted(glob.glob(os.path.join(data_dir,"file_*.csv")))
    items = {}
    for path in files:
        base = os.path.basename(path)
        m = re.match(r"file_(\d+)\.csv", base)
        if not m:
            continue
        item_id = int(m.group(1))
        try:
            df = pd.read_csv(path)
        except Exception as e:
            print(f"Skip {base}: read error {e}")
            continue
        if not {"v","zct"}.issubset(df.columns):
            print(f"Skip {base}: missing v/zct")
            continue
        df = df[["v","zct"]].apply(pd.to_numeric, errors="coerce")
        # Clean zct
        zct = df["zct"].dropna().values
        if len(zct) >= 2:
            zct = np.sort(np.unique(zct))
            if len(zct) >= 5:
                zct = pd.Series(zct).rolling(5, center=True, min_periods=1).median().values
            diffs = np.diff(zct)
            keep = [0] + [i+1 for i in range(len(diffs)) if diffs[i] > 0]
            zct = zct[keep]
        if len(zct) < 2:
            # Fallback: create coarse cycle boundaries across the recording
            n = len(df["v"])
            approx_cycles = max(2, int(n / FS))
            zct = np.linspace(0.0, approx_cycles, approx_cycles+1)
        items[item_id] = {"v": df["v"].dropna().values.astype(float),
                          "zct": zct.astype(float),
                          "filename": base}
    print(f"Loaded {len(items)} files")
    return items

# ============================
# Tachometer/turbine utilities
# ============================
def tachometer_frequency(zct):
    delta = np.diff(zct)
    delta = np.where(delta <= 0, np.nan, delta)
    f = 1.0/(delta)
    return f

def turbine_speed(f_tach):
    return f_tach * GEAR_RATIO

# ============================
# Band energy feature (for anomaly clustering)
# ============================
def band_energy(signal, fs, center, bw=50.0):
    nyq = 0.5*fs
    low_hz = max(1.0, center - bw)
    high_hz = min(nyq - 1.0, center + bw)
    if low_hz >= high_hz:
        return 0.0
    low = low_hz/nyq
    high = high_hz/nyq
    try:
        b,a = butter(4,[low,high],btype='band')
        filt = filtfilt(b,a,signal)
        return float(np.mean(filt**2))
    except Exception:
        return 0.0

# ============================
# Per-file summary features (V and ZCT)
# ============================
def summarize_file(v, zct, fs=FS):
    # V features (acceleration)
    v_rms = float(np.sqrt(np.mean(v**2))) if len(v) > 0 else 0.0
    v_skew = float(skew(v)) if len(v) > 10 else 0.0
    v_kurt = float(kurtosis(v)) if len(v) > 10 else 0.0
    # ZCT features (tach)
    f_tach = tachometer_frequency(zct)
    tach_mean = float(np.nanmean(f_tach)) if len(f_tach) else 0.0
    tach_std  = float(np.nanstd(f_tach))  if len(f_tach) else 0.0
    turbine_mean = float(np.nanmean(turbine_speed(f_tach))) if len(f_tach) else 0.0

    # Optional bearing band energies from whole-record signal
    be_cage  = band_energy(v, fs, FAULT_BANDS["cage"],  bw=40.0)
    be_ball  = band_energy(v, fs, FAULT_BANDS["ball"],  bw=80.0)
    be_inner = band_energy(v, fs, FAULT_BANDS["inner"], bw=80.0)
    be_outer = band_energy(v, fs, FAULT_BANDS["outer"], bw=80.0)

    return {
        "v_rms": v_rms, "v_skew": v_skew, "v_kurt": v_kurt,
        "tach_mean": tach_mean, "tach_std": tach_std, "turbine_mean": turbine_mean,
        "band_cage": be_cage, "band_ball": be_ball, "band_inner": be_inner, "band_outer": be_outer
    }

def build_summary(items):
    rows = []
    for item_id in sorted(items.keys()):
        v = items[item_id]["v"]; zct = items[item_id]["zct"]
        s = summarize_file(v, zct, fs=FS)
        s["item_id"] = item_id
        rows.append(s)
    summary = pd.DataFrame(rows).sort_values("item_id")
    summary.to_csv(os.path.join(DATA_DIR, "summary_v_zct.csv"), index=False)
    return summary

# ============================
# Pairwise mapping between files (combined V–ZCT)
# ============================
def pairwise_vzct(summary):
    ids = summary["item_id"].values.tolist()
    # Feature vector combining V and ZCT summaries
    F = summary[["v_rms","v_skew","v_kurt","tach_mean","tach_std","turbine_mean"]].values.astype(float)
    # Robust z-normalization across items
    med = np.median(F, axis=0)
    madv = 1.4826 * (np.median(np.abs(F - med), axis=0) + 1e-12)
    Fz = (F - med) / madv
    # Pairwise L1 distance matrix
    n = len(ids)
    M = np.zeros((n,n), dtype=float)
    for i in range(n):
        for j in range(n):
            M[i,j] = float(np.sum(np.abs(Fz[i] - Fz[j])))
    dfM = pd.DataFrame(M, index=ids, columns=ids)
    dfM.to_csv(os.path.join(DATA_DIR, "pairwise_map_vzct.csv"))
    # Deviation score per item: L1 distance from the robust center (zero vector)
    dev_scores = np.sum(np.abs(Fz), axis=1)
    dev_df = pd.DataFrame({"item_id": ids, "deviation_score": dev_scores})
    dev_df = dev_df.sort_values("item_id")
    dev_df.to_csv(os.path.join(DATA_DIR, "deviation_scores.csv"), index=False)
    return dfM, dev_df, Fz

# ============================
# Relational graphs across files (presented vs bell-curve order)
# ============================
def plot_relational_graphs(summary, dev_df):
    # Presented order: file_01 -> file_53
    ids_presented = summary["item_id"].values
    y_v = summary["v_rms"].values
    y_z = summary["tach_mean"].values

    fig, ax = plt.subplots(2, 1, figsize=(12,8), sharex=True)
    ax[0].plot(ids_presented, smooth_centered(y_v, 9), color='steelblue', lw=1.5, label='V RMS (smoothed)')
    ax[0].plot(ids_presented, y_v, color='lightblue', lw=0.5, alpha=0.5)
    ax[0].set_title("Presented order (file_01 → file_53): V RMS")
    ax[0].set_ylabel("V RMS")
    ax[0].legend()

    ax[1].plot(ids_presented, smooth_centered(y_z, 9), color='darkorange', lw=1.5, label='Tach mean (smoothed)')
    ax[1].plot(ids_presented, y_z, color='moccasin', lw=0.5, alpha=0.5)
    ax[1].set_title("Presented order: Tachometer mean frequency")
    ax[1].set_xlabel("Item id (presented order)")
    ax[1].set_ylabel("Tach mean (Hz)")
    ax[1].legend()

    fig.tight_layout()
    fig.savefig(os.path.join(PLOT_DIR, "relational_presented.png"))
    plt.close(fig)

    # Bell-curve order: sort by deviation_score ascending (center → tails)
    dev_sorted = dev_df.sort_values("deviation_score")
    ids_bell = dev_sorted["item_id"].values
    # Map summary to bell order
    s_idx = {iid: i for i, iid in enumerate(ids_presented)}
    y_v_bell = np.array([summary.loc[s_idx[iid], "v_rms"] for iid in ids_bell])
    y_z_bell = np.array([summary.loc[s_idx[iid], "tach_mean"] for iid in ids_bell])

    fig, ax = plt.subplots(2, 1, figsize=(12,8), sharex=True)
    ax[0].plot(range(1,len(ids_bell)+1), smooth_centered(y_v_bell, 9), color='seagreen', lw=1.5, label='V RMS (smoothed)')
    ax[0].plot(range(1,len(ids_bell)+1), y_v_bell, color='palegreen', lw=0.5, alpha=0.5)
    ax[0].set_title("Bell-curve order (by deviation from normal): V RMS")
    ax[0].set_ylabel("V RMS")
    ax[0].legend()

    ax[1].plot(range(1,len(ids_bell)+1), smooth_centered(y_z_bell, 9), color='purple', lw=1.5, label='Tach mean (smoothed)')
    ax[1].plot(range(1,len(ids_bell)+1), y_z_bell, color='thistle', lw=0.5, alpha=0.5)
    ax[1].set_title("Bell-curve order: Tachometer mean frequency")
    ax[1].set_xlabel("Rank in bell-curve order (center → tails)")
    ax[1].set_ylabel("Tach mean (Hz)")
    ax[1].legend()

    fig.tight_layout()
    fig.savefig(os.path.join(PLOT_DIR, "relational_bell.png"))
    plt.close(fig)

    # Scatter comparisons (V vs ZCT) in both orders
    fig, ax = plt.subplots(1, 2, figsize=(14,6))
    ax[0].scatter(y_v, y_z, c=ids_presented, cmap='viridis', s=30)
    ax[0].set_title("Presented order: V RMS vs Tach mean")
    ax[0].set_xlabel("V RMS"); ax[0].set_ylabel("Tach mean (Hz)")

    ax[1].scatter(y_v_bell, y_z_bell, c=range(len(ids_bell)), cmap='plasma', s=30)
    ax[1].set_title("Bell-curve order: V RMS vs Tach mean")
    ax[1].set_xlabel("V RMS"); ax[1].set_ylabel("Tach mean (Hz)")

    fig.tight_layout()
    fig.savefig(os.path.join(PLOT_DIR, "scatter_v_vs_zct.png"))
    plt.close(fig)

# ============================
# Anomaly clustering (engineered features per file)
# ============================
def anomaly_score_for_order(summary):
    # Use robust z of V RMS and band energies to emulate anomaly clustering
    cols = ["v_rms","band_cage","band_ball","band_inner","band_outer"]
    F = summary[cols].values.astype(float)
    med = np.median(F, axis=0)
    madv = 1.4826 * (np.median(np.abs(F - med), axis=0) + 1e-12)
    Fz = (F - med) / madv
    # Aggregate anomaly magnitude
    A = np.sum(np.abs(Fz), axis=1)
    ids = summary["item_id"].values
    order = [i for i,_ in sorted(zip(ids, A), key=lambda kv: kv[1])]  # low anomaly first
    return order, A

# ============================
# Final order fusion and submission
# ============================
def ranks_from_order(order):
    return {item_id: rank for rank, item_id in enumerate(order, start=1)}

def fuse_orders(bell_order, anomaly_order):
    ids = sorted(set(bell_order).union(set(anomaly_order)))
    r_bell = ranks_from_order(bell_order)
    r_anom = ranks_from_order(anomaly_order)
    # Favor anomaly clustering to emulate 134288, but include bell-curve relational progression
    fused_score = {i: 0.6 * r_anom.get(i, len(ids)) + 0.4 * r_bell.get(i, len(ids)) for i in ids}
    final = [i for i,_ in sorted(fused_score.items(), key=lambda kv: kv[1])]
    return final

# ============================
# Main pipeline
# ============================
def run_pipeline_and_submit(data_dir=DATA_DIR, out_path="submission.csv"):
    # Load files
    items = load_files(data_dir)
    if not items:
        raise RuntimeError("No item CSVs loaded. Ensure /content contains file_XX.csv with v,zct.")

    # Build V–ZCT summary
    summary = build_summary(items)

    # Pairwise mapping and deviation scores
    dfM, dev_df, Fz = pairwise_vzct(summary)
    print("Saved pairwise_map_vzct.csv and deviation_scores.csv")

    # Relational graphs across files (presented vs bell-curve)
    plot_relational_graphs(summary, dev_df)
    print(f"Saved relational graphs to {PLOT_DIR}")

    # Bell-curve order (center-to-tails by deviation from normal)
    bell_order = dev_df.sort_values("deviation_score")["item_id"].tolist()

    # Anomaly-clustered order (bearing-feature emphasis)
    anom_order, A = anomaly_score_for_order(summary)

    # Fuse into final prediction
    final_order = fuse_orders(bell_order, anom_order)

    # Write submission
    sub_df = pd.DataFrame({"prediction": final_order})
    sub_df.to_csv(out_path, index=False)
    print(f"Wrote: {out_path}")
    print("First 20 predictions:", final_order[:20])

# ============================
# Entrypoint
# ============================
if __name__ == "__main__":
    run_pipeline_and_submit(DATA_DIR, out_path="submission.csv")

Loaded 53 files
Saved pairwise_map_vzct.csv and deviation_scores.csv
Saved relational graphs to /content/relational_graphs
Wrote: submission.csv
First 20 predictions: [9, 11, 4, 12, 1, 10, 18, 7, 6, 17, 2, 3, 19, 16, 5, 23, 26, 8, 30, 27]
